### Imports

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

#for macbook
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Device:", device)


Device: mps


### Configs

In [ ]:
DATA_DIR = Path("train_images/train_images")
csv_path = Path("train_images.csv") 

batch_size = 32
learning_rate = 0.0005 #old value: 0.0001
max_lr = 0.003
weight_decay = 0.0004 
num_epochs = 80
num_classes = 200
val_split = 0.2
seed = 42

torch.manual_seed(seed)
np.random.seed(seed)
print("Device:", device)


if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("⚡ Running on Apple MPS GPU")
else:
    device = torch.device("cpu")
    print("🐌 Running on CPU")

Device: mps
⚡ Running on Apple MPS GPU


### Datapreperation

### Dataset class

In [5]:
class BirdDataset(Dataset):
    def __init__(self, csv_file, root_dir, img_col_idx, label_col_idx, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.img_col_idx = img_col_idx
        self.label_col_idx = label_col_idx
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename = str(self.data.iloc[idx, self.img_col_idx])
        clean_filename = filename.lstrip('/').lstrip('\\')
        img_path = os.path.join(self.root_dir, clean_filename)

        try:
            image = Image.open(img_path).convert('RGB')
        except (FileNotFoundError, OSError):
            print(f"Could not open {img_path}, using black image.")
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        # PyTorch labels: 0..199
        raw_label = int(self.data.iloc[idx, self.label_col_idx])
        label = raw_label - 1

        if self.transform:
            image = self.transform(image)

        return image, label
    
    
    

### Transformers

In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.02),
    transforms.RandomAffine(degrees=15, translate=(0.1,0.1), scale=(0.8,1.2), shear=10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
    transforms.RandomErasing(p=0.15)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])

### Train validation split

In [7]:
full_train_dataset = BirdDataset('train_images.csv', 'train_images', 0, 1, transform=train_transform)
full_val_dataset   = BirdDataset('train_images.csv', 'train_images', 0, 1, transform=val_transform)
labels = full_train_dataset.data.iloc[:, 1].values - 1

skf = StratifiedKFold(n_splits=int(1/val_split), shuffle=True, random_state=seed)
for train_idx, val_idx in skf.split(np.zeros(len(labels)), labels):
    break

train_dataset = Subset(full_train_dataset, train_idx)
val_dataset   = Subset(full_val_dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

test_dataset  = BirdDataset('test_images_path.csv', 'test_images', 1, 2, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Test set: {len(test_dataset)} images")

Train: 3140, Val: 786
Test set: 4000 images


### Model setup

In [8]:
class BirdNet(nn.Module):
    def __init__(self, num_classes=200, num_attributes=0):
        super().__init__()
        self.backbone = models.resnet18(weights=None)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        self.dropout = nn.Dropout(p=0.5) #added
        self.classifier = nn.Linear(in_features, num_classes)
        
        if num_attributes > 0:
            self.attribute_head = nn.Linear(in_features, num_attributes)
        else:
            self.attribute_head = None

    def forward(self, x):
        feats = self.backbone(x)
        
        feats = self.dropout(feats) #added
        
        if self.attribute_head:
            return self.classifier(feats), self.attribute_head(feats)
        else:
            return self.classifier(feats)

### Training + TensorBoard Logging nieuw

In [ ]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim

model = BirdNet(num_classes=num_classes).to(device)
print("Model reset")

#settings
best_acc = 0
writer = SummaryWriter(log_dir="runs/ResNet18_cnn_param_lr_wd") 

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Training Loop
for epoch in range(1, num_epochs+1):
    model.train()
    total_loss, correct, total = 0, 0, 0

    # Progress bar per epoch
    loop = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} Training", leave=False)
    for batch_idx, (imgs, labels) in enumerate(loop):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(imgs)
        if isinstance(outputs, tuple):
            outputs = outputs[0]  

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

        # Batch-level TensorBoard
        step = (epoch - 1) * len(train_loader) + batch_idx
        writer.add_scalar("Loss/train_batch", loss.item(), step)
        writer.add_scalar("Accuracy/train_batch", (outputs.argmax(1) == labels).float().mean().item(), step)

        # Update tqdm postfix
        loop.set_postfix(loss=loss.item(), acc=(outputs.argmax(1) == labels).float().mean().item())

    train_acc = correct / total
    train_loss = total_loss / total

    # Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    val_loss = val_loss / total

    # Epoch-level TensorBoard
    writer.add_scalar("Loss/train_epoch", train_loss, epoch)
    writer.add_scalar("Loss/val_epoch", val_loss, epoch)
    writer.add_scalar("Accuracy/train_epoch", train_acc, epoch)
    writer.add_scalar("Accuracy/val_epoch", val_acc, epoch)
    writer.add_scalar("LR", scheduler.get_last_lr()[0], epoch)

    print(f"Epoch {epoch}/{num_epochs} | "
          f"Train Acc: {train_acc:.4f}, Train Loss: {train_loss:.4f} | "
          f"Val Acc: {val_acc:.4f}, Val Loss: {val_loss:.4f}")

    scheduler.step()

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model_ResNet18_cnn_param_lr_wd.pth")
        print("New best model saved")

writer.close()

Model gereset: we beginnen weer vanaf nul.


Epoch 1/80 | Train Acc: 0.0127, Train Loss: 5.4578 | Val Acc: 0.0140, Val Loss: 5.2207
✔️ New best model saved! 🎉


Epoch 2/80 | Train Acc: 0.0162, Train Loss: 5.1795 | Val Acc: 0.0293, Val Loss: 5.0023
✔️ New best model saved! 🎉


Epoch 3/80 | Train Acc: 0.0220, Train Loss: 5.0677 | Val Acc: 0.0369, Val Loss: 4.9672
✔️ New best model saved! 🎉


Epoch 4/80 | Train Acc: 0.0280, Train Loss: 4.9971 | Val Acc: 0.0420, Val Loss: 4.9244
✔️ New best model saved! 🎉


...

Epoch 76/80 | Train Acc: 0.7939, Train Loss: 1.8216 | Val Acc: 0.3537, Val Loss: 3.4182


Epoch 77/80 | Train Acc: 0.7965, Train Loss: 1.7980 | Val Acc: 0.3372, Val Loss: 3.4379


Epoch 78/80 | Train Acc: 0.8025, Train Loss: 1.7935 | Val Acc: 0.3511, Val Loss: 3.4126


Epoch 79/80 | Train Acc: 0.8121, Train Loss: 1.7824 | Val Acc: 0.3422, Val Loss: 3.4344


Epoch 80/80 | Train Acc: 0.8105, Train Loss: 1.7852 | Val Acc: 0.3461, Val Loss: 3.4116


### Test and submission 


In [ ]:
# Loading model
model = BirdNet(num_classes=num_classes, num_attributes=0).to(device)
checkpoint_path = "best_model_ResNet18_cnn_param_lr_wd.pth"

if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=False)
    print(f"Loaded model weights from: {checkpoint_path}")
else:
    print(f"file not found: {checkpoint_path}")

model.eval()

# Predictions
predictions = []
print("Starting predictions...")

with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Predicting"):
        inputs = inputs.to(device)
        outputs = model(inputs)
        
        if isinstance(outputs, tuple):
            outputs = outputs[0]
            
        _, preds = torch.max(outputs, 1)
        predictions.extend((preds.cpu().numpy() + 1))

# save as csv
submission = pd.DataFrame({
    "id": range(1, len(predictions) + 1),
    "label": predictions
})

output_file = "submissions_ResNet18_cnn_param_lr_wd.csv"
submission.to_csv(output_file, index=False)

print(f"Subimmision ready: {output_file}")
print(submission.head())

✅ Model weights geladen (extra layers genegeerd) van: best_model_ResNet18_cnn_param_lr_wd.pth
Starting predictions...


Predicting: 100%|██████████| 125/125 [00:34<00:00,  3.63it/s]

✅ Klaar! Bestand opgeslagen als: submissions_ResNet18_cnn_param_lr_wd.csv
   id  label
0   1     67
1   2    118
2   3     78
3   4     28
4   5     74
